In [8]:
import numpy as np
import libpysal
import spreg
import geopandas as gpd
import polars as pl
from local_functions import *

In [5]:
territorial_information = gpd.read_file("data/transformed/territorial_information_summary.geojson")

In [3]:
for c in territorial_information.columns:
    print(c)

Teritoriālā vienība
Platība (km^2)
Gads
Iedzīvotāju skaits gada sākumā
Gada vidējais iedzīvotāju skaits
Iedzīvotāju dabiskais pieaugums
Migrācijas saldo
Iedzīvotāju skaita izmaiņas - pavisam
Mēneša darba samaksas mediāna
Iedzīvotāji darbspējas vecumā
Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)
Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)
Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Izgl

In [21]:
y_col = ['Mēneša darba samaksas mediāna']
x_col = [
    'Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Izglītība (Kapitālie izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi) (uz 1 iedz.)',
    'Ceļu un ielu izdevumi (investīcijas) (uz 1 iedz.)',
    'Ekonomiski aktīvi uzņēmumi (uz 1000 iedz.)',
    'Iedzīvotāju īpatsvars ar augstāko izglītību',
    'Iedzīvotāji darbspējas vecumā (%)'
]

columns_to_use = [
    'Teritoriālā vienība',
    'Gads',
    'geometry'
] + x_col + y_col

In [22]:
(
    pl.from_dataframe(territorial_information.drop("geometry", axis=1))
    .unpivot(
        on=y_col + x_col,
        index=['Teritoriālā vienība', 'Gads']
    )
    .filter(~pl.col("value").is_null())
    .group_by("variable")
    .agg(
        pl.col("Gads").min().alias("Min year"),
        pl.col("Gads").max().alias("Max year")
    )
    .group_by(1)
    .agg(
        pl.col("Min year").max(),
        pl.col("Max year").min()
    )
)

literal,Min year,Max year
i32,i32,i32
1,2021,2023


In [23]:
df = territorial_information[columns_to_use]
df = df[df.Gads.isin([2021, 2022, 2023])]
df = df.fillna(0)
df = df.sort_values(by=['Teritoriālā vienība', 'Gads'])

In [24]:
t = df[df.Gads==2023]
w = libpysal.weights.Queen.from_dataframe(t, ids="Teritoriālā vienība")
w.transform = 'r'

In [25]:
df[x_col]

,Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Izglītība (Kapitālie izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi) (uz 1 iedz.),"Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi) (uz 1 iedz.)",Ceļu un ielu izdevumi (investīcijas) (uz 1 iedz.),Ekonomiski aktīvi uzņēmumi (uz 1000 iedz.),Iedzīvotāju īpatsvars ar augstāko izglītību,Iedzīvotāji darbspējas vecumā (%)
323,650.161474,79.583069,185.103347,55.233289,35.861852,5.205673,67.0,21.3,0.621378
366,628.043332,48.336396,80.784374,59.237653,13.401859,76.220306,69.0,21.9,0.616452
409,786.987807,17.310674,95.787599,81.883282,7.224629,69.284318,68.0,22.1,0.612850
301,553.604574,261.387490,49.767838,29.226463,26.693673,25.794748,77.0,21.5,0.616478
344,619.892862,702.290296,236.518729,396.929878,63.190754,184.962395,82.0,22.2,0.615617
...,...,...,...,...,...,...,...,...,...
375,734.185335,353.122676,0.465230,165.775611,6.927497,79.774152,88.0,46.2,0.621764
418,827.251988,54.610572,2.347291,89.275539,3.962520,27.032163,85.0,46.8,0.621029
338,578.944077,310.798617,110.941450,10.035176,7.565482,80.876384,81.0,39.8,0.619044
381,655.583391,100.792055,93.966395,16.169432,3.392428,41.494519,86.0,40.8,0.618141


In [26]:
Y_df = df[y_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[y_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [27]:
X_df = df[x_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[x_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [28]:
Y = Y_df.to_numpy()
X = X_df.to_numpy()

fe_lag = spreg.Panel_FE_Lag(Y, X, w, name_y = y_col, name_x=x_col, name_ds="df")

In [29]:
print(fe_lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG PANEL - FIXED EFFECTS
-----------------------------------------------------------------------
Data set            :          df
Weights matrix      :     unknown
Dependent Variable  :Mēneša darba samaksas mediāna                Number of Observations:         129
Mean dependent var  :      0.0000                Number of Variables   :          10
S.D. dependent var  :     98.8127                Degrees of Freedom    :         119
Pseudo R-squared    :      0.9933
Spatial Pseudo R-squared:  0.9879
Sigma-square ML     :      64.912                Log likelihood        :    -780.727
S.E of regression   :       8.057                Akaike info criterion :    1581.454
                                                 Schwarz criterion     :    1610.052

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
